# Assignment: CSCA 5632 Unsupervised Algorithms for Customer Segmentation and Recommendation Systems

## Introduction

E-commerce businesses generate vast amounts of transaction data, but extracting meaningful insights from this data remains challenging. Key business goals include understanding customer behavior, providing personalized product recommendations, and predicting future purchases. Addressing these challenges can improve marketing effectiveness, increase customer retention, and boost sales.

In this assignment, I focus on applying unsupervised machine learning techniques to analyze real-world e-commerce data. The main objectives are:

1. Customer segmentation to distinct groups of customers based on their purchasing behavior.  
2. Product recommendation to suggest relevant items to customers to enhance cross-selling and upselling opportunities.

By exploring these patterns, I aim to demonstrate how data-driven insights can inform better business decisions in online retail.

## Dataset Description

- Dataset: UCI Online Retail Dataset  
- Source: https://archive.ics.uci.edu/dataset/352/online+retail  
- Business Context: Transactions from a UK-based online retailer specializing in gifts, including both individual and wholesale customers.  
- Timeframe: December 2010 – December 2011  
- Size: 541,909 transactions, 8 attributes

This dataset provides a realistic snapshot of e-commerce operations, capturing diverse purchasing behaviors across a range of products.

## Problem Definition

E-commerce businesses collect large volumes of transaction data, but turning this data into actionable insights is not straightforward. Two key challenges are particularly important: understanding different customer groups and making personalized product recommendations.

Without effective segmentation, marketing campaigns tend to be generic, leading to low engagement and wasted advertising budgets. Similarly, poor recommendation systems can result in missed cross-selling opportunities, fewer repeat purchases, and higher customer churn.

In this project, I aim to use unsupervised learning methods, such as clustering based on purchase history and customer behavior, to identify meaningful customer segments. I expect that understanding these segments will help deliver better recommendations, improve customer lifetime value, and increase sales.

## Why I Chose This Dataset

I chose this dataset because it represents real-world e-commerce data, rather than a simplified or synthetic dataset. This means it includes imperfections, inconsistencies, and missing values, which provide an opportunity to practice data cleaning, transformation, and exploratory analysis in a realistic context.

Its richness and complexity make it well-suited for unsupervised learning techniques such as clustering, dimensionality reduction, and matrix factorization. These methods can uncover hidden patterns in customer behavior, purchasing preferences, and product relationships. Working with this dataset allows me to extract practical insights that closely mirror real business scenarios and challenges.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score


# Online Retail Data Loading

It loads the **Online Retail** dataset from an Excel file into a Pandas DataFrame.


In [ ]:
import os
data_path = 'online_retail.xlsx'

df = pd.read_excel(data_path)
display(df.head())
print(df.dtypes)


# Dataset Overview

This shows a quick summary of the dataset using Pandas.


In [ ]:

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
print(f"Unique customers: {df['Customer ID'].nunique()}")
print(f"Unique products: {df['Description'].nunique()}")
print(f"Unique countries: {df['Country'].nunique()}")

# Missing Data Analysis

This code computes the missing values in each column of the dataset and shows missing counts and percentages.


In [ ]:
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

# Exploratory Data Analysis: Online Retail Dataset

This exploratory data analysis gives an overview of customer behavior, product popularity, and sales trends in the Online Retail dataset. Most purchases involve moderate quantities and affordable unit prices, while sales are concentrated in a few key countries. Clear monthly and weekday patterns emerge, with distinct peak hours during the day. A small number of products drive most of the purchases, and revenue distribution is uneven, showing the presence of high-value customers. Overall, customer engagement varies widely, with some buyers being highly active and others purchasing only occasionally.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use("seaborn-v0_8")
fig, axes = plt.subplots(3, 3, figsize=(24, 14))  # wider and shorter
axes = axes.flatten()

# ---  Quantity distribution ---
quantity_filtered = df[df['Quantity'].between(0, 100)]['Quantity']
axes[0].hist(quantity_filtered, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Quantity Distribution (0–100)', fontsize=14)
axes[0].set_xlabel('Quantity')
axes[0].set_ylabel('Frequency')

# ---  Price distribution ---
price_filtered = df[df['Price'].between(0, 50)]['Price']
axes[1].hist(price_filtered, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Price Distribution (0–50)', fontsize=14)
axes[1].set_xlabel('Price (£)')
axes[1].set_ylabel('Frequency')

# ---  Top countries ---
top_countries = df['Country'].value_counts().head(10)
top_countries.plot(kind='bar', ax=axes[2], color="skyblue", edgecolor="black")
axes[2].set_title('Top 10 Countries by Transactions', fontsize=14)
axes[2].set_xlabel('Country')
axes[2].set_ylabel('Transaction Count')
axes[2].tick_params(axis='x', rotation=45)

# ---  Monthly sales trend ---
df['YearMonth'] = pd.to_datetime(df['InvoiceDate']).dt.to_period('M')
monthly_sales = df.groupby('YearMonth')['Price'].sum()
monthly_sales.plot(kind='line', marker='o', ax=axes[3], color="darkblue")
axes[3].set_title('Monthly Sales Trend', fontsize=14)
axes[3].set_xlabel('Month')
axes[3].set_ylabel('Total Sales (£)')
axes[3].tick_params(axis='x', rotation=45)

# ---  Customer transaction frequency ---
customer_transactions = df.groupby('Customer ID')['Invoice'].nunique()
axes[4].hist(customer_transactions, bins=50, edgecolor='black', alpha=0.7, color="orange")
axes[4].set_title('Customer Transaction Frequency', fontsize=14)
axes[4].set_xlabel('Transactions per Customer')
axes[4].set_ylabel('Number of Customers')

# ---  Top products ---
top_products = df['Description'].value_counts().head(15)
top_products.plot(kind='barh', ax=axes[5], color="teal")
axes[5].set_title('Top 15 Most Popular Products', fontsize=14)
axes[5].set_xlabel('Purchase Count')
axes[5].set_ylabel('Product')
axes[5].tick_params(axis='y', labelsize=9)  # smaller font for long names

# ---  Revenue per customer ---
customer_revenue = df.groupby('Customer ID')['Price'].sum()
axes[6].hist(customer_revenue[customer_revenue <= 1000], bins=50, edgecolor='black', alpha=0.7, color="purple")
axes[6].set_title('Customer Revenue Distribution (≤£1000)', fontsize=14)
axes[6].set_xlabel('Revenue (£)')
axes[6].set_ylabel('Customers')

# ---  Weekday analysis ---
df['Weekday'] = pd.to_datetime(df['InvoiceDate']).dt.day_name()
weekday_sales = df.groupby('Weekday')['Price'].sum()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_sales = weekday_sales.reindex(weekday_order)
weekday_sales.plot(kind='bar', ax=axes[7], color="salmon", edgecolor="black")
axes[7].set_title('Sales by Weekday', fontsize=14)
axes[7].set_xlabel('Day of Week')
axes[7].set_ylabel('Total Sales (£)')
axes[7].tick_params(axis='x', rotation=30)

# ---  Hourly transactions ---
df['Hour'] = pd.to_datetime(df['InvoiceDate']).dt.hour
hourly_transactions = df.groupby('Hour').size()
hourly_transactions.plot(kind='line', marker='o', ax=axes[8], color="green")
axes[8].set_title('Transactions by Hour', fontsize=14)
axes[8].set_xlabel('Hour of Day')
axes[8].set_ylabel('Transactions')

plt.subplots_adjust(
    top=0.95, 
    bottom=0.05, 
    left=0.05, 
    right=0.98, 
    hspace=0.35, 
    wspace=0.25
)
plt.show()


## Data Cleaning

The dataset required thorough cleaning to ensure reliable analysis. This involved removing cancelled orders, invalid quantities and prices, missing or duplicate customer records, and incorrect dates. All columns were standardized, and the data was reindexed to create a consistent and accurate dataset for further analysis.

In [ ]:
import pandas as pd

df_clean = df.copy()

# Remove cancelled invoices (InvoiceNo starting with 'C')
df_clean = df_clean[~df_clean['Invoice'].astype(str).str.startswith('C')]

# Remove negative or zero Quantity and Price
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['Price'] > 0)]

# Drop rows with missing CustomerID
df_clean = df_clean.dropna(subset=['Customer ID'])

# Drop duplicates
df_clean = df_clean.drop_duplicates()

# Ensure InvoiceDate is datetime and handle errors
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], errors='coerce')
df_clean = df_clean.dropna(subset=['InvoiceDate'])  # Remove rows where date conversion failed
df_clean['Customer ID'] = df_clean['Customer ID'].astype('int64')


df_clean = df_clean.reset_index(drop=True)


In [ ]:
print("Original shape:", df.shape)
print("Cleaned shape :", df_clean.shape)

In [ ]:
print("\n--- Missing Values After Cleaning ---")
print(df_clean.isnull().sum())


In [ ]:
print("--- Sample Clean Data ---")
display(df_clean.head())

## Data Cleaning Summary


In [ ]:
print("Data cleaning analysis:")
print(f"Original dataset: {df.shape[0]:,} rows")
print(f"After cleaning: {df_clean.shape[0]:,} rows")
print(f"Removed: {df.shape[0] - df_clean.shape[0]:,} rows ({(df.shape[0] - df_clean.shape[0])/df.shape[0]*100:.1f}%)")

print(f"\nQuality checks after cleaning:")
print(f"- Missing Customer IDs: {df_clean['Customer ID'].isnull().sum()}")
print(f"- Negative quantities: {(df_clean['Quantity'] < 0).sum()}")
print(f"- Zero/negative prices: {(df_clean['Price'] <= 0).sum()}")
print(f"- Duplicate rows: {df_clean.duplicated().sum()}")

## Cleaned Data Statistics

The cleaned dataset includes 4,312 unique customers and 4,444 unique products, spanning the period from December 2009 to December 2010. Total revenue amounts to £1.33 million, with an average transaction value of £3.31.

In [ ]:
print("--- CLEANED DATA CHARACTERISTICS ---")
print(f"Unique customers: {df_clean['Customer ID'].nunique():,}")
print(f"Unique products: {df_clean['Description'].nunique():,}")
print(f"Date range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Total revenue: {df_clean['Price'].sum():,.2f}")
print(f"Average transaction value: {df_clean['Price'].mean():.2f}")

## Customer Metrics Overview

After cleaning and removing outliers, the dataset includes 4,268 customers. The following visualization highlights the distribution of recency, frequency, and monetary metrics, with RFM for short. Skewed metrics were log-transformed to normalize their distributions, which is important for clustering methods that rely on distance measures. Recency was not log-transformed, as it does not exhibit a long-tailed distribution.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score

ref_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Reference date for recency: {ref_date}")

rfm = df_clean.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (ref_date - x.max()).days,  # Recency (days)
    'Invoice': 'nunique',                                # Frequency (unique invoices)
    'Price': 'sum'                                       # Monetary (total spent)
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

customer_metrics = df_clean.groupby('Customer ID').agg({
    'Quantity': ['sum', 'mean'],                         # Total and average quantity
    'Description': 'nunique',                            # Product variety
    'InvoiceDate': lambda x: (x.max() - x.min()).days   # Customer lifetime span
}).reset_index()

customer_metrics.columns = ['CustomerID', 'TotalQuantity', 'AvgQuantity', 'ProductVariety', 'LifetimeSpan']
rfm = rfm.merge(customer_metrics, on='CustomerID')

print(f"Before outlier removal: {len(rfm)} customers")
rfm = rfm[(rfm['Monetary'] > 0) & (rfm['Frequency'] > 0)]
rfm = rfm[rfm['Monetary'] <= rfm['Monetary'].quantile(0.99)]  # Remove top 1% to reduce skewness
print(f"After outlier removal: {len(rfm)} customers")

plt.figure(figsize=(20, 12))

for i, col in enumerate(['Recency', 'Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety']):
    plt.subplot(3, 5, i+1)
    sns.histplot(rfm[col], kde=True, bins=30)
    plt.title(f'{col} Distribution')
    
    # Add statistical annotations
    mean_val = rfm[col].mean()
    median_val = rfm[col].median()
    plt.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.1f}')
    plt.axvline(median_val, color='green', linestyle='--', label=f'Median: {median_val:.1f}')
    plt.legend(fontsize=8)

rfm_log = rfm.copy()
for col in ['Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety']:
    rfm_log[col] = np.log1p(rfm_log[col])

# Log-transformed distributions
for i, col in enumerate(['Recency', 'Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety']):
    plt.subplot(3, 5, i+6)
    sns.histplot(rfm_log[col], kde=True, bins=30)
    plt.title(f'{col} (Log-transformed)')

plt.tight_layout()
plt.show()

print(rfm[['Recency', 'Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety']].describe())


## RFM Feature Correlation Analysis

The correlation analysis of RFM features reveals several notable patterns in customer behavior. There is a strong positive correlation between monetary value and product variety (r = 0.88), indicating that customers who spend more also tend to purchase a wider range of products. Frequency and monetary value are moderately correlated (r = 0.66), suggesting that customers who make purchases more often also tend to spend more overall. Additionally, frequency and product variety show a positive correlation (r = 0.55), meaning that frequent buyers are more likely to explore and purchase from a broader selection of products. These insights highlight the relationships between how often customers buy, how much they spend, and the diversity of their purchases.

In [ ]:

correlation_matrix = rfm[['Recency', 'Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    cmap='coolwarm', 
    center=0, 
    fmt='.2f',
    linewidths=0.5,
    cbar=True
)
plt.title('Feature Correlations', fontsize=14, pad=12)
plt.tight_layout()
plt.show()


## PC Analysis
Reducing the RFM features to 2 dimensions using PCA serves a few important purposes. First, it simplifies the data while retaining most of the original variability, making it easier to visualize and interpret customer patterns. Working in a lower-dimensional space also removes correlations between features, which helps clustering algorithms like K-Means perform more effectively because they rely on distance measures. Finally, reducing to two dimensions allows us to plot the customers on a 2D scatter, providing an intuitive view of the clusters and helping validate the quality of the segmentation visually. Overall, PCA makes the clustering process faster, more robust, and easier to interpret without losing key information.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

feature_cols = ['Recency', 'Frequency', 'Monetary']
X = rfm_log[feature_cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
rfm['PCA1'] = X_pca[:, 0]
rfm['PCA2'] = X_pca[:, 1]

explained_variance = pca.explained_variance_ratio_
print(f"Explained variance by each component: {explained_variance}")
print(f"Total variance explained by 2 components: {explained_variance.sum():.2f}")

pca_components = pd.DataFrame(
    pca.components_,
    columns=feature_cols,
    index=['PCA1', 'PCA2']
)
pca_components = pca_components.round(3) 

print("\nPCA Composition Table:")
display(pca_components)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.6, edgecolor='k')
plt.xlabel(f'PCA1 ({explained_variance[0]*100:.1f}% variance)')
plt.ylabel(f'PCA2 ({explained_variance[1]*100:.1f}% variance)')
plt.title('2D PCA of RFM Features')
plt.grid(True)
plt.show()



# K-Means Clustering for Customers segmentation

K-Means clustering was applied to the scaled RFM features, with the number of clusters (k) varied from 2 to 10 in a parameter sweep. The clustering results were evaluated using Inertia, Silhouette Score, and Davies-Bouldin Index to assess cluster compactness and separation. Based on these metrics, 3 clusters were identified as the optimal choice, providing a good balance between cohesion within clusters and separation between clusters. Visualizations of the clusters further confirm the quality of this segmentation and support the selection of the optimal

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

# --- K-Means clustering hyperparameter sweeping---
inertia, silhouette, davies_bouldin = [], [], []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20, max_iter=300)
    labels = kmeans.fit_predict(X_pca)
    
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(X_pca, labels))
    davies_bouldin.append(davies_bouldin_score(X_pca, labels))
    
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, "
          f"Silhouette={silhouette_score(X_pca, labels):.3f}, "
          f"Davies-Bouldin={davies_bouldin_score(X_pca, labels):.3f}")

norm_inertia = [(max(inertia)-i)/(max(inertia)-min(inertia)) for i in inertia]
norm_silhouette = [(s - min(silhouette))/(max(silhouette)-min(silhouette)) for s in silhouette]
norm_db = [(max(davies_bouldin)-db)/(max(davies_bouldin)-min(davies_bouldin)) for db in davies_bouldin]
combined_score = [sum(x) for x in zip(norm_inertia, norm_silhouette, norm_db)]
optimal_k = K[combined_score.index(max(combined_score))]
print(f"\nOptimal number of clusters: {optimal_k}")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(K, inertia, 'bo-', markersize=8, linewidth=2)
axes[0, 0].set_title("Elbow Method")
axes[0, 0].set_xlabel("k")
axes[0, 0].set_ylabel("Inertia")

axes[0, 1].plot(K, silhouette, 'ro-', markersize=8, linewidth=2)
axes[0, 1].set_title("Silhouette Score")
axes[0, 1].set_xlabel("k")
axes[0, 1].set_ylabel("Score")

axes[1, 0].plot(K, davies_bouldin, 'go-', markersize=8, linewidth=2)
axes[1, 0].set_title("Davies-Bouldin Index (lower better)")
axes[1, 0].set_xlabel("k")
axes[1, 0].set_ylabel("DB Index")

axes[1, 1].plot(K, combined_score, 'mo-', markersize=8, linewidth=2)
axes[1, 1].axvline(optimal_k, color='red', linestyle='--', label=f'Optimal k={optimal_k}')
axes[1, 1].set_title("Combined Score")
axes[1, 1].set_xlabel("k")
axes[1, 1].set_ylabel("Normalized Score")
axes[1, 1].legend()

plt.tight_layout()
plt.show()


# Final K-Means Clustering (k=3)

The final K-Means clustering model (k=3) produced a silhouette score of 0.390, indicating moderate cluster separation. The clusters are distributed fairly evenly.

Cluster 0 represents customers with high recency but low frequency and spending, suggesting recent but low-value engagement. Cluster 1 includes highly active and valuable customers, with high frequency, monetary value, and product variety, reflecting loyal and engaged buyers. Cluster 2 captures customers with moderate recency, low to moderate frequency, and spending, representing occasional or moderately engaged customers. These clusters provide insights into customer behavior, helping guide targeted marketing and retention strategies.


In [ ]:
k_final = 3
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=20, max_iter=300, algorithm='lloyd')
rfm['Cluster'] = kmeans_final.fit_predict(X_pca)

final_silhouette = silhouette_score(X_scaled, rfm['Cluster'])
print(f"Final K-means model performance:")
print(f"- Silhouette score: {final_silhouette:.3f}")

cluster_counts = rfm['Cluster'].value_counts().sort_index()
print("Cluster distribution:")
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count:,} customers ({count/len(rfm)*100:.1f}%)")

cluster_profiles = rfm.groupby('Cluster')[['Recency','Frequency','Monetary','TotalQuantity','ProductVariety','LifetimeSpan']].agg(['mean','median','std']).round(2)
display(cluster_profiles)



# Final K-Means Cluster Visualization 


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

sns.scatterplot(x='Recency', y='Monetary', hue='Cluster', palette='Set2', data=rfm, ax=axes[0, 0], s=60, alpha=0.7)
axes[0, 0].set_title("K-Means (k=3): Recency vs Monetary")
axes[0, 0].set_xlabel("Recency (days)")
axes[0, 0].set_ylabel("Monetary (£)")

sns.scatterplot(x='Frequency', y='Monetary', hue='Cluster', palette='Set2', data=rfm, ax=axes[0, 1], s=60, alpha=0.7)
axes[0, 1].set_title("K-Means (k=3): Frequency vs Monetary")
axes[0, 1].set_xlabel("Frequency (transactions)")
axes[0, 1].set_ylabel("Monetary (£)")

rfm_melted = pd.melt(rfm[['Cluster','Recency','Frequency','Monetary']], id_vars=['Cluster'], var_name='RFM_Dimension', value_name='Value')
sns.boxplot(data=rfm_melted[rfm_melted['RFM_Dimension']=='Recency'], x='Cluster', y='Value', ax=axes[1, 0], palette='Set2', hue='Cluster')
axes[1, 0].set_title("K-Means (k=3): Recency by Cluster")
axes[1, 0].set_ylabel("Recency (days)")

sns.boxplot(data=rfm_melted[rfm_melted['RFM_Dimension']=='Monetary'], x='Cluster', y='Value', ax=axes[1, 1], palette='Set2', hue='Cluster')
axes[1, 1].set_title("K-Means (k=3): Monetary by Cluster")
axes[1, 1].set_ylabel("Monetary (£)")

plt.tight_layout()
plt.show()


# K-Means Clustering Visualization with centroid (k=3) 


| Cluster | Recency       | Frequency      | Monetary Value | Product Variety | Description                              |
|---------|---------------|----------------|----------------|----------------|------------------------------------------|
| 0       | High          | Low            | Low            | Low            | low-value customers           |
| 1       | Medium/Low    | High           | High           | High           | Loyal and highly engaged customers       |
| 2       | Moderate      | Low to Moderate| Low to Moderate| Moderate       | moderately engaged customers|



In [ ]:

centroids_pca = kmeans_final.cluster_centers_

plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=rfm, x='PCA1', y='PCA2',
    hue='Cluster', palette='Set2',
    s=60, alpha=0.6
)

plt.scatter(
    centroids_pca[:, 0], centroids_pca[:, 1],
    c='black', marker='X', s=250, label='Centroids'
)

for i, (x, y) in enumerate(centroids_pca):
    plt.text(x + 0.1, y + 0.1, f'Cluster {i}', fontsize=12, weight='bold')

plt.title(f'K-Means Clusters in 2D PCA Space (k={optimal_k})', fontsize=16)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


# Agglomerative Clustering for customer segmentation

Agglomerative clustering was applied to PCA-reduced RFM features (2 components) for 4,268 customers, testing 3–7 clusters with ward, complete, average, and single linkages. The best result was 3 clusters with Ward linkage, achieving a silhouette score of around 0.41. Ward linkage produced compact, well-separated clusters, and the silhouette decline with more clusters indicates potential over-segmentation. Selecting 3 clusters aligns with standard RFM segmentation (High, Medium, Low value), balancing statistical separation and business interpretability.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


n_clusters_range = range(3, 8)
linkage_methods = ['ward', 'complete', 'average', 'single']

results = []

for n_clusters in n_clusters_range:
    for linkage_method in linkage_methods:
        metric = 'euclidean' if linkage_method == 'ward' else 'manhattan'
        
        agglo = AgglomerativeClustering(
            n_clusters=n_clusters,
            linkage=linkage_method,
            metric=metric
        )
        labels = agglo.fit_predict(X_pca)
        
        sil = silhouette_score(X_pca, labels)
        results.append({
            'n_clusters': n_clusters,
            'linkage': linkage_method,
            'silhouette': sil
        })
        print(f"Clusters={n_clusters}, Linkage={linkage_method} -> Silhouette={sil:.3f}")

results_df = pd.DataFrame(results)

best_idx = results_df['silhouette'].idxmax()
best_config = results_df.loc[best_idx]
print("\nBest Agglomerative Clustering configuration:")
print(best_config)


# Agglomerative Clustering visualziation

Agglomerative clustering on PCA-reduced RFM features produced 3 clusters using Ward linkage: Cluster 0 with 2,086 customers (48.9%), Cluster 1 with 1,097 (25.7%), and Cluster 2 with 1,085 (25.4%), achieving a silhouette score of 0.41. Ward linkage generated compact, well-separated clusters, with reasonably balanced sizes that align with typical RFM segmentation (High, Medium, Low value). The silhouette score slightly outperformed K-Means (0.39), highlighting differences between hierarchical and centroid-based approaches.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, normalized_mutual_info_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

n_clusters = 3
linkage_method = "ward"

agglo = AgglomerativeClustering(
    n_clusters=n_clusters, 
    linkage=linkage_method, 
    metric='euclidean'
)
rfm['Agglo_Cluster'] = agglo.fit_predict(X_pca)

print(f"Agglomerative clustering with {n_clusters} clusters completed")
agglo_counts = rfm['Agglo_Cluster'].value_counts().sort_index()
for cluster, count in agglo_counts.items():
    percentage = count / len(rfm) * 100
    print(f"  Cluster {cluster}: {count:,} customers ({percentage:.1f}%)")

agglo_silhouette = silhouette_score(X_pca, rfm['Agglo_Cluster'])
agglo_db = davies_bouldin_score(X_pca, rfm['Agglo_Cluster'])
nmi = normalized_mutual_info_score(rfm['Cluster'], rfm['Agglo_Cluster'])

print(f"\nAgglomerative clustering performance:")
print(f"- Silhouette score: {agglo_silhouette:.3f}")


# --- Cluster Profile Summary ---
cluster_profiles = (
    rfm.groupby('Agglo_Cluster')[['Recency', 'Frequency', 'Monetary', 'TotalQuantity', 'ProductVariety', 'LifetimeSpan']]
    .agg(['mean', 'median', 'std'])
    .round(2)
)

print("\nCluster Profile Summary:")
display(cluster_profiles)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=rfm['Agglo_Cluster'], palette='Set2', s=60, alpha=0.7)
plt.title(f"Agglomerative Clustering ({n_clusters} clusters) on PCA-reduced data")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend(title='Cluster')
plt.grid(alpha=0.3)
plt.show()


# Visualizing Agglomerative Clustering Results

The plots show the distribution of Recency, Frequency, and Monetary values across the 3 agglomerative clusters. Scatterplots highlight how clusters separate in key RFM dimensions, while boxplots reveal cluster-specific distributions and central tendencies for Recency and Monetary value.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sample_data = X_scaled if len(rfm) <= 1000 else X_scaled[np.random.choice(len(rfm), 1000, replace=False)]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

sns.scatterplot(
    x='Recency', y='Monetary', hue='Agglo_Cluster', palette='Set2', 
    data=rfm, ax=axes[0, 0], s=60, alpha=0.7
)
axes[0, 0].set_title("Recency vs Monetary (Agglomerative)")
axes[0, 0].set_xlabel("Recency (days)")
axes[0, 0].set_ylabel("Monetary (£)")

sns.scatterplot(
    x='Frequency', y='Monetary', hue='Agglo_Cluster', palette='Set2', 
    data=rfm, ax=axes[0, 1], s=60, alpha=0.7
)
axes[0, 1].set_title("Frequency vs Monetary (Agglomerative)")
axes[0, 1].set_xlabel("Frequency (transactions)")
axes[0, 1].set_ylabel("Monetary (£)")

rfm_melted = pd.melt(
    rfm[['Agglo_Cluster','Recency','Frequency','Monetary']], 
    id_vars=['Agglo_Cluster'], var_name='RFM_Dimension', value_name='Value'
)
sns.boxplot(
    data=rfm_melted[rfm_melted['RFM_Dimension']=='Recency'], 
    x='Agglo_Cluster', y='Value', ax=axes[1, 0], palette='Set2', hue='Agglo_Cluster'
)
axes[1, 0].set_title("Recency by Agglomerative Cluster")
axes[1, 0].set_ylabel("Recency (days)")

sns.boxplot(
    data=rfm_melted[rfm_melted['RFM_Dimension']=='Monetary'], 
    x='Agglo_Cluster', y='Value', ax=axes[1, 1], palette='Set2', hue='Agglo_Cluster'
)
axes[1, 1].set_title("Monetary by Agglomerative Cluster")
axes[1, 1].set_ylabel("Monetary (£)")

plt.tight_layout()
plt.show()



# K-Means vs Agglomerative Clustering: Summary & Comparison

K-Means and Agglomerative clustering both produced three meaningful customer segments. The silhouette score for Agglomerative clustering (0.410) was slightly higher than for K-Means (0.390), indicating better cluster cohesion. Based on mean RFM values, the clusters can be described as follows:

- High-value customers: low Recency, high Frequency and Monetary 
- Medium-value customers: moderate Recency and Monetary 
- Low-value customers: high Recency, low Frequency and Monetary

K-Means is fast and works well for compact, well-separated clusters because it assumes spherical shapes. Agglomerative clustering does not assume specific cluster shapes and shows hierarchical relationships between customers, giving more insight into nested or irregular patterns. Overall, both methods identify similar customer patterns and can be used for targeted marketing: keeping high-value customers, re-engaging medium-value customers, and trying to revive low-value customers. Agglomerative clustering provides slightly better separation and more structural insight.





In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage


print(f"- K-Means Silhouette: {final_silhouette:.3f}")
print(f"- Agglomerative Silhouette: {agglo_silhouette:.3f}")

# --- K-Means cluster profiles ---
kmeans_profiles = rfm.groupby('Cluster')[['Recency','Frequency','Monetary']].mean().round(2)
print("K-Means Cluster Profiles (mean RFM values):")
print(kmeans_profiles)

# --- Agglomerative cluster profiles ---
agglo_profiles = rfm.groupby('Agglo_Cluster')[['Recency','Frequency','Monetary']].mean().round(2)
print("\nAgglomerative Cluster Profiles (mean RFM values):")
print(agglo_profiles)
fig, axes = plt.subplots(1, 5, figsize=(25, 5))
(ax1, ax2, ax3, ax4, ax5) = axes.flatten()

# K-Means results
ax1.scatter(rfm['PCA1'], rfm['PCA2'], c=rfm['Cluster'], cmap='Set2', alpha=0.7, s=50)
ax1.set_title('K-Means Clustering')
ax1.set_xlabel('PCA1')
ax1.set_ylabel('PCA2')

# Agglomerative results
ax2.scatter(rfm['PCA1'], rfm['PCA2'], c=rfm['Agglo_Cluster'], cmap='Set2', alpha=0.7, s=50)
ax2.set_title(f'Agglomerative Clustering\n({linkage_method}, k={n_clusters})')
ax2.set_xlabel('PCA1')
ax2.set_ylabel('PCA2')

# Dendrogram
sample_data = X_scaled if len(rfm) <= 1000 else X_scaled[np.random.choice(len(rfm), 1000, replace=False)]
linkage_matrix = linkage(sample_data, method='ward')
dendrogram(linkage_matrix, truncate_mode='level', p=5, ax=ax3)
ax3.set_title('Hierarchical Dendrogram')
ax3.set_xlabel('Sample Index')
ax3.set_ylabel('Distance')

# Silhouette comparison
methods = ['K-Means', 'Agglomerative']
silhouette_scores = [final_silhouette, agglo_silhouette]
bars = ax4.bar(methods, silhouette_scores, color=['skyblue', 'lightcoral'])
ax4.set_title('Silhouette Score Comparison')
ax4.set_ylabel('Silhouette Score')
ax4.set_ylim(0, max(silhouette_scores) * 1.1)
for bar, score in zip(bars, silhouette_scores):
    ax4.text(bar.get_x() + bar.get_width()/2., score + 0.01, f'{score:.3f}', ha='center', va='bottom')

# Cluster size comparison
cluster_sizes_kmeans = rfm['Cluster'].value_counts().sort_index()
cluster_sizes_agglo = rfm['Agglo_Cluster'].value_counts().sort_index()
x = np.arange(len(cluster_sizes_kmeans))
width = 0.35

ax5.bar(x - width/2, cluster_sizes_kmeans.values, width, label='K-Means', color='skyblue')
ax5.bar(x + width/2, cluster_sizes_agglo.values, width, label='Agglomerative', color='lightcoral')

ax5.set_title('Cluster Size Comparison')
ax5.set_xlabel('Cluster')
ax5.set_ylabel('Number of Customers')
ax5.set_xticks(x)

# Use the actual cluster indices for labels
ax5.set_xticklabels([f'Cluster {i}' for i in cluster_sizes_kmeans.index])
ax5.legend()


## Collaborative Filtering Recommendation System

To further explore on additional unsupervised machine learning algo, I implemented a collaborative filtering recommendation system using both user-based and item-based approaches on the Online Retail dataset. The customer-product matrix contained 4312 customers and 4444 products, with a sparsity of 98.5%. To handle the wide range of purchase quantities, values were log-scaled using np.log1p.

The system computes cosine similarities for both customers and products. For item-based CF, a shrinkage factor is applied to adjust for co-purchase counts, reducing noise from items with few interactions.

User-based CF generates predictions by weighting purchases of the 20 most similar customers, centering by each customer's mean to account for different purchasing scales. Item-based CF predicts scores by summing contributions from previously purchased items weighted by similarity.

The dataset was split into training and test sets, masking 20% of interactions for testing. RMSE evaluation on the test set gave the following results:

| Method     | RMSE    |
|------------|---------|
| User-CF    | 2.0217  |
| Item-CF    | 19.8311 |

This indicate that user-based CF provides more accurate predictions in this setting, likely because item-based CF struggles with the high sparsity and limited co-purchase information. Overall, collaborative filtering demonstrates the ability to generate relevant recommendations and can be used to enhance customer experience and suggest new products.







In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

class CFRecommendationSystem:
    
    def __init__(self, customer_product_matrix: pd.DataFrame, shrinkage_lambda=10):
        self.customer_product_matrix = np.log1p(customer_product_matrix)
        self.customer_similarity = None
        self.product_similarity = None
        self.shrinkage_lambda = shrinkage_lambda
        
    def compute_similarities(self):
        self.customer_similarity = cosine_similarity(self.customer_product_matrix)
        
        raw_item_similarity = cosine_similarity(self.customer_product_matrix.T)
        
        bin_matrix = (self.customer_product_matrix > 0).astype(float).T.values
        co_counts = bin_matrix @ bin_matrix.T
        
        self.product_similarity = raw_item_similarity * co_counts / (co_counts + self.shrinkage_lambda)
        np.fill_diagonal(self.product_similarity, 1.0)
    
    def predict_user_cf(self, customer_id: int) -> np.ndarray:
        if customer_id not in self.customer_product_matrix.index:
            return np.zeros(self.customer_product_matrix.shape[1])
        
        customer_idx = self.customer_product_matrix.index.get_loc(customer_id)
        similarities = self.customer_similarity[customer_idx]
        
        similar_customers_idx = similarities.argsort()[-21:][::-1][1:]
        weights = similarities[similar_customers_idx]
        
        if weights.sum() == 0:
            return np.zeros(self.customer_product_matrix.shape[1])
        
        similar_customers_purchases = self.customer_product_matrix.iloc[similar_customers_idx]
        
        mean_user = self.customer_product_matrix.iloc[customer_idx].mean()
        mean_neighbors = similar_customers_purchases.mean(axis=1)
        centered_neighbors = similar_customers_purchases.sub(mean_neighbors, axis=0)
        
        pred = mean_user + (centered_neighbors.T * weights).sum(axis=1) / weights.sum()
        return np.clip(pred.values, 0, None)
    
    def predict_item_cf(self, customer_id: int) -> np.ndarray:
        if customer_id not in self.customer_product_matrix.index:
            return np.zeros(self.customer_product_matrix.shape[1])
        
        customer_purchases = self.customer_product_matrix.loc[customer_id]
        purchased_items = customer_purchases[customer_purchases > 0].index
        
        if len(purchased_items) == 0:
            return np.zeros(len(customer_purchases))
        
        scores = pd.Series(0.0, index=self.customer_product_matrix.columns)
        for item in purchased_items:
            item_idx = self.customer_product_matrix.columns.get_loc(item)
            scores += self.product_similarity[item_idx] * customer_purchases[item]
        
        return np.clip(scores.values, 0, None)
    
    def evaluate_rmse(self, test_matrix: pd.DataFrame, method='user') -> float:
        predictions = []
        actuals = []
        
        test_matrix_scaled = np.log1p(test_matrix)
        
        for customer_id in self.customer_product_matrix.index:
            if method == 'user':
                pred = self.predict_user_cf(customer_id)
            else:
                pred = self.predict_item_cf(customer_id)
            
            actual = test_matrix_scaled.loc[customer_id].values
            mask = actual > 0
            if mask.sum() > 0:
                predictions.extend(pred[mask])
                actuals.extend(actual[mask])
        
        rmse = np.sqrt(mean_squared_error(actuals, predictions))
        return rmse



customer_product_matrix = df_clean.pivot_table(
    index='Customer ID',
    columns='Description',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

np.random.seed(42)
matrix = customer_product_matrix.values.copy()
test_mask = np.random.rand(*matrix.shape) < 0.2

train_matrix = matrix.copy()
train_matrix[test_mask] = 0

test_matrix = matrix.copy()
test_matrix[~test_mask] = 0

train_df = pd.DataFrame(train_matrix, index=customer_product_matrix.index, columns=customer_product_matrix.columns)
test_df = pd.DataFrame(test_matrix, index=customer_product_matrix.index, columns=customer_product_matrix.columns)

# Train CF Recommendataion system
cf_system = CFRecommendationSystem(train_df)
cf_system.compute_similarities()

user_rmse = cf_system.evaluate_rmse(test_df, method='user')
item_rmse = cf_system.evaluate_rmse(test_df, method='item')

print(f"User-CF RMSE: {user_rmse:.4f}")
print(f"Item-CF RMSE: {item_rmse:.4f}")


This example shows recommendations for a sample customer (ID: 12346). The already purchased items include mostly doormats and a few umbrellas and parasols. 

The top 10 user-based CF recommendations suggest items that similar customers have purchased, introducing some new products like hanging heart t-light holders and ceramic trinket boxes. 

The top 10 item-based CF recommendations focus on products similar to those already bought, reinforcing familiar items like union flag doormats and hearts doormats, along with related home accessories.


In [ ]:
# Sample customer
sample_customer_id = train_df.index[0]
top_n = 10

# --- Already purchased items ---
purchased_mask = train_df.loc[sample_customer_id] > 0
purchased_items = train_df.columns[purchased_mask]

# --- User-CF recommendations ---
user_pred = cf_system.predict_user_cf(sample_customer_id)
user_pred_masked = np.where(purchased_mask, -np.inf, user_pred)
top_user_idx = np.argsort(user_pred_masked)[-top_n:][::-1]
user_recommended_items = train_df.columns[top_user_idx]

# --- Item-CF recommendations ---
item_pred = cf_system.predict_item_cf(sample_customer_id)
item_pred_masked = np.where(purchased_mask, -np.inf, item_pred)
top_item_idx = np.argsort(item_pred_masked)[-top_n:][::-1]
item_recommended_items = train_df.columns[top_item_idx]

# --- Display results ---
print(f"Customer ID: {sample_customer_id}\n")

print("Already purchased items:")
for i, item in enumerate(purchased_items, 1):
    print(f"{i}. {item}")

print("\nTop User-CF recommended items:")
for i, item in enumerate(user_recommended_items, 1):
    print(f"{i}. {item}")

print("\nTop Item-CF recommended items:")
for i, item in enumerate(item_recommended_items, 1):
    print(f"{i}. {item}")


This heatmap visualizes the pairwise cosine similarity between a sample of 20 customers. Higher values (warmer colors) indicate more similar purchase behavior, helping to identify clusters of customers with similar preferences for user-based recommendations.








In [ ]:
sample_idx = list(range(20))
plt.figure(figsize=(10, 8))
sns.heatmap(cf_system.customer_similarity[np.ix_(sample_idx, sample_idx)],
            cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Customer Similarity Heatmap (Sample 20 Customers)")
plt.show()


This heatmap shows the similarity between the top 20 most purchased products. Warmer colors indicate higher similarity, reflecting items that are frequently bought together and guiding item-based recommendations.


In [ ]:
top_products = customer_product_matrix.sum().sort_values(ascending=False).index[:20]
product_idx = [customer_product_matrix.columns.get_loc(p) for p in top_products]

plt.figure(figsize=(10, 8))
sns.heatmap(cf_system.product_similarity[np.ix_(product_idx, product_idx)],
            xticklabels=top_products, yticklabels=top_products,
            cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Product Similarity Heatmap (Top 20 Products)")
plt.show()


### NMF Latent Factors & Sales Forecast

To extend it further to try if can use NMF for this online sales dataset, I refer to one paper from here https://www.jstage.jst.go.jp/article/ipsjjip/27/0/27_752/_pdf, and applied NMF to the Online Retail dataset to extract latent factors for products and customers. First, the purchase quantities were aggregated into a tensor of shape `(products × customers × months)` and log-scaled using `np.log1p`. Averaging over the time dimension produced a `(products × customers)` matrix for NMF.

A hyperparameter sweep was conducted over latent dimensions from 5 to 30. As the number of components increased, the reconstruction mean squared error (MSE) decreased, indicating better approximation of the original data. For example, with 5 latent factors the MSE was 0.000952, while with 30 factors it dropped to 0.000708. This helps select the optimal number of latent factors for capturing patterns in the data.

The resulting factor matrices consist of product latent factors (`U`) of shape `(num_products × K)` and customer latent factors (`V`) of shape `(K × num_customers)`. Each row of `U` represents a product’s strength across latent patterns such as home décor or kitchenware, while each column of `V` represents a customer’s preference for each latent factor.

Forecasting can be performed linearly over time for each element of the latent matrices, producing predicted matrices `U_forecast` and `V_forecast`. The final reconstruction `X_pred = U_forecast @ V_forecast` gives a `(num_products × num_customers)` matrix that estimates the likelihood of a customer purchasing a product in the future. This approach allows for capturing temporal trends and hidden patterns in customer-product interactions.

This is quite interesting to implement NMF to make the sales forcasting


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['Month'] = df_clean['InvoiceDate'].dt.to_period('M')

customers = df_clean['Customer ID'].unique()
products = df_clean['StockCode'].unique()
months = df_clean['Month'].sort_values().unique()
T = len(months)

customer_to_idx = {c: i for i, c in enumerate(customers)}
product_to_idx = {p: i for i, p in enumerate(products)}
month_to_idx = {m: i for i, m in enumerate(months)}

df_clean['CustomerIdx'] = df_clean['Customer ID'].map(customer_to_idx)
df_clean['ProductIdx'] = df_clean['StockCode'].map(product_to_idx)
df_clean['MonthIdx'] = df_clean['Month'].map(month_to_idx)

# --- Build tensor Z (product x customer x month) ---
Z = np.zeros((len(products), len(customers), T), dtype=np.float32)
indices = df_clean[['ProductIdx', 'CustomerIdx', 'MonthIdx']].values
quantities = df_clean['Quantity'].values
Z[indices[:, 0], indices[:, 1], indices[:, 2]] = quantities

Z = np.log1p(Z)

# --- Average over time dimension ---
XAve = Z.mean(axis=2)  # product x customer matrix

# --- hyperparameter sweep for latent factors ---
component_list = [5, 10, 15, 20, 25, 30, 35, 40]
results = []

for K in component_list:
    try:
        nmf_model = NMF(
            n_components=K,
            init='nndsvda',
            random_state=42,
            max_iter=500
        )
        U = nmf_model.fit_transform(XAve)
        V = nmf_model.components_
        X_reconstructed = U @ V
        mse = mean_squared_error(XAve, X_reconstructed)
        results.append({'n_components': K, 'MSE': mse})
        print(f"K={K} -> MSE: {mse:.6f}")
    except Exception as e:
        print(f"K={K} failed: {e}")

df_results = pd.DataFrame(results)
print("\hyperparameter Sweep Results:")
print(df_results)

# --- Plot MSE vs n_components ---
plt.figure(figsize=(6, 4))
plt.plot(df_results['n_components'], df_results['MSE'], marker='o')
plt.xlabel('Latent Dimensions (n_components)')
plt.ylabel('Reconstruction MSE')
plt.title('NMF Parameter Sweep')
plt.grid(True)
plt.tight_layout()
plt.show()




## Choosing NMF Components and Recommendation Threshold

For NMF, the MSE steadily decreases as `n_components` increases, but the rate of improvement slows down after around 20–25 components. To balance reconstruction accuracy and model simplicity, I have chosen n_components = 20.

The recommendation threshold is currently set at 0.8 to reduce false positives, meaning that only the most confident predictions are recommended. 

## Future Improvement

In future improvements, the threshold could be determined using the 95th percentile of predicted scores, which would select the top 5% of predictions and may lead to better overall recommendation performance.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['Month'] = df_clean['InvoiceDate'].dt.to_period('M')

customers = df_clean['Customer ID'].unique()
products = df_clean['StockCode'].unique()
months = df_clean['Month'].sort_values().unique()
T = len(months)

customer_to_idx = {c: i for i, c in enumerate(customers)}
product_to_idx = {p: i for i, p in enumerate(products)}
month_to_idx = {m: i for i, m in enumerate(months)}

df_clean['CustomerIdx'] = df_clean['Customer ID'].map(customer_to_idx)
df_clean['ProductIdx'] = df_clean['StockCode'].map(product_to_idx)
df_clean['MonthIdx'] = df_clean['Month'].map(month_to_idx)

# --- Create 3D tensor: products x customers x months ---
Z = np.zeros((len(products), len(customers), T), dtype=np.float32)
indices = df_clean[['ProductIdx','CustomerIdx','MonthIdx']].values
quantities = df_clean['Quantity'].values
Z[indices[:,0], indices[:,1], indices[:,2]] = quantities

Z = np.log1p(Z) 

XAve = Z.mean(axis=2)  # average over time to get a static product-customer matrix
K = 20
nmf_model = NMF(n_components=K, init='nndsvda', random_state=42, max_iter=500)
Uinit = nmf_model.fit_transform(XAve)  # customer latent factors
Vinit = nmf_model.components_          # product latent factors

def linear_forecast(matrix_seq):
    T, dim1, dim2 = matrix_seq.shape
    x = np.arange(T)
    forecast = np.zeros((dim1, dim2), dtype=np.float32)
    
    for i in range(dim1):
        for j in range(dim2):
            y = matrix_seq[:, i, j]
            mask = y != 0
            if mask.sum() > 1:
                coeff = np.polyfit(x[mask], y[mask], 1) 
                forecast[i, j] = coeff[0]*T + coeff[1] 
            elif mask.sum() == 1:
                forecast[i, j] = y[mask][0]
    return np.maximum(forecast, 0)

U_forecast = linear_forecast(np.broadcast_to(Uinit, (T, *Uinit.shape)))
V_forecast = linear_forecast(np.broadcast_to(Vinit, (T, *Vinit.shape)))

X_pred = U_forecast @ V_forecast
threshold = 0.8
X_pred_binary = (X_pred > threshold).astype(np.int8)

prod_idx, cust_idx = np.nonzero(X_pred_binary)
df_pred = pd.DataFrame({
    'CustomerID': customers[cust_idx],
    'ProductID': products[prod_idx],
    'Predicted': 1
})

print(df_pred.shape)
print(df_pred.head(5))


## Heatmap of XAve

This heatmap displays a subset of the XAve matrix (average log-quantity) for 100 products and 100 customers. Darker shades correspond to higher average purchase quantities, emphasizing the most frequently bought products and the most active customers in XAve.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sample_customers = 100
sample_products = 100
XAve_sample = XAve[:sample_products, :sample_customers]

plt.figure(figsize=(12, 8))
sns.heatmap(XAve_sample, cmap='YlGnBu', cbar_kws={'label': 'Average Log-Quantity'})
plt.xlabel('Customers')
plt.ylabel('Products')
plt.title('Heatmap of Average Product-Customer Matrix (XAve)')
plt.show()


## Heatmap of Predicted Purchases

This heatmap visualizes the predicted customer-product purchases from df_pred. Cells with darker blue indicate a higher likelihood that the customer will purchase the product, highlighting the model's top recommendations.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_heat = df_pred.pivot(index='CustomerID', columns='ProductID', values='Predicted').fillna(0)

plt.figure(figsize=(15,8))
sns.heatmap(df_heat, cmap='Blues', cbar_kws={'label':'Predicted Purchase'})
plt.title("Predicted Customer-Product Purchases")
plt.xlabel("Product")
plt.ylabel("Customer")
plt.show()


# Shows all predicted products for Customer 14911

In [ ]:
customer_14911_pred = df_pred[df_pred['CustomerID'] == 14911]
print(customer_14911_pred)


## Summary

This project was my attempt to apply unsupervised machine learning techniques to the UCI Online Retail Dataset. It was quite challenging at first, but I learned a lot along the way. I started by cleaning the messy dataset, removing about 25% of bad data, which was more than I expected. After that, I explored customer purchasing patterns through visualizations and used RFM analysis to understand customer behavior in terms of recency, frequency, and monetary value. To handle the high dimensionality of the data, I applied PCA for dimensionality reduction. I then tried both K-Means and Agglomerative clustering to segment customers into three groups. Additionally, I built collaborative filtering recommendation systems, though the data was extremely sparse, with 98.5% missing values.

The key findings from this project were insightful. I identified three distinct customer segments: high-value loyal customers, moderate occasional buyers, and low-value at-risk customers. Between the clustering methods, Agglomerative clustering performed slightly better than K-Means, achieving a silhouette score of 0.41 compared to 0.39. In the recommendation system, user-based collaborative filtering performed much better than item-based, with an RMSE of 2.02 versus 19.83. Overall, the clustering produced clear and interpretable groups, which was helpful for understanding customer behavior. However, the recommendation task was challenging due to the high sparsity of the data, as most customers purchased only a few items.

## Conclusion

This project taught me that real-world data is often messy and difficult to work with. While I was able to successfully identify customer segments that make business sense, I realized there is still a lot more to learn about handling sparse data and selecting the most suitable algorithms. I learned that data preprocessing is far more important than I initially thought, and that different clustering algorithms can provide different insights into the same dataset. Sparsity is a major issue in recommendation systems, and visualization plays a critical role in understanding how algorithms operate and what patterns they capture.

## Next Step
For the next steps, I explored NMF for forecasting, which I found particularly interesting. I implemented NMF on a 3D tensor of products, customers, and months, then averaged over time to extract latent factors. Inspired by a paper I read, this approach worked reasonably well, with MSE decreasing from 0.000952 using 5 factors to 0.000708 with 30 factors. I believe there is still much more to explore in this area, including optimizing the latent factors and experimenting with different tensor factorization techniques.

# reference
https://www.jstage.jst.go.jp/article/ipsjjip/27/0/27_752/_pdf